In [1]:
%%writefile mapper_tf.py

import sys
import re

from collections import Counter

lookup_word = "labor"

path = 'stop_words_en.txt'

def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))

stop_words = read_stop_words(path)


for line in sys.stdin:
    try:
        article_id, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue

    words = [w.lower() for w in re.split("\W*\s+\W*", text, flags=re.UNICODE) if w.lower() not in stop_words]
    total_words = len(words)
    word_count = Counter(words)
    if lookup_word in word_count:
        tf = word_count[lookup_word] / total_words
        print("%s\t%f\t%s" % (lookup_word, tf, article_id))

Overwriting mapper_tf.py


In [2]:
%%writefile reducer_tfidf.py

import sys

from math import log

lookup_article_id = 12

lookup_tf = None
word_sum = 0

for line in sys.stdin:
    try:
        lookup_word, tf, article_id = line.strip().split('\t')
    except ValueError as e:
        continue
    word_sum += 1
    if int(article_id) == lookup_article_id:
        lookup_tf = float(tf)


if lookup_tf:
    idf = 1. / log(1 + word_sum)
    print ("%0.6f" % (lookup_tf * idf))

Overwriting reducer_tfidf.py


In [3]:
%%bash

OUT_DIR="tfidf_result"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

# Stub code for your job

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_tf.py,reducer_tfidf.py,/datasets/stop_words_en.txt \
    -mapper "python3 mapper_tf.py" \
    -reducer "python3 reducer_tfidf.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-*

0.000351	


21/11/01 16:08:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/11/01 16:08:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/11/01 16:08:49 INFO mapred.FileInputFormat: Total input files to process : 1
21/11/01 16:08:49 INFO mapreduce.JobSubmitter: number of splits:2
21/11/01 16:08:49 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
21/11/01 16:08:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1635782503509_0002
21/11/01 16:08:50 INFO conf.Configuration: resource-types.xml not found
21/11/01 16:08:50 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
21/11/01 16:08:50 INFO resource.ResourceUtils: Adding resource type - name = memory-mb, units = Mi, type = COUNTABLE
21/11/01 16:08:50 INFO resource.ResourceUtils: Adding resource type - name = vcores, units = , type = COUNTABLE
21/11/01 16:08:50 INFO impl.Ya